In this script, we apply the best ensemble strategy found in ensemble_boston_test.ipynb to predict on Toronto images.

The strategy is an averaging + weighted ensemble:

1. average the predictions produced by cv3 and full

2. average the predictions produced by cv0, cv1,cv2, cv4 and own

3. give prediction by model 'cv3full' 0.45 weight; and prediction by model 'cv0cv1cv2cv4own' 0.55 weight


In [3]:
import pandas as pd
import numpy as np
import glob

#### Data preparation

As toronto_prediction_own_cnn.csv and the other predictions were produced in 2 different VMs, we need to unify them first.

toronto_prediction_own_cnn.csv has predictions for 2034 images, whereas the other csv have predictions for 2038 images. That is because in 2 VMs, the number of images fetched by the Google Street View API were not the same. So we removed that 4 extra rows in the other csv.

In [35]:
predict_dir = '/Users/zhanglingling/Desktop/ML1030/toronto_prediction/'
file_list = list(glob.glob(predict_dir + "*.csv*"))
file_list.sort()
file_list

['/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.hdf5.cv0.prediction.csv',
 '/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.hdf5.cv1.prediction.csv',
 '/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.hdf5.cv2.prediction.csv',
 '/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.hdf5.cv3.prediction.csv',
 '/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.hdf5.cv4.prediction.csv',
 '/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.wholedata.hdf5.prediction.csv',
 '/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_prediction_own_cnn.csv']

In [9]:
# p = pd.read_csv('/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_prediction_own_cnn.csv')     
# cv4 = pd.read_csv('/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.hdf5.cv4.prediction.csv')
# df = pd.merge(p, cv4, how='right', on='_file', 
#                     indicator=True)
# df[df['_merge'] == 'right_only']

,_file,0_x,1_x,0_y,1_y,_merge
2034,gsv_1343.jpg,NaN,NaN,0.083244,0.916756,right_only
2035,gsv_1075.jpg,NaN,NaN,0.930137,0.069863,right_only
2036,gsv_52.jpg,NaN,NaN,0.993009,0.006991,right_only
2037,gsv_1407.jpg,NaN,NaN,0.999982,0.000018,right_only


In [24]:
# # remove the 4 rows in the predictions by cv0, cv1, cv2, cv3, cv4 and full 
# cv0 = cv0[~cv0['_file'].isin(['gsv_52.jpg', 'gsv_1075.jpg', 'gsv_1343.jpg', 'gsv_1407.jpg'])]
# cv1 = cv1[~cv1['_file'].isin(['gsv_52.jpg', 'gsv_1075.jpg', 'gsv_1343.jpg', 'gsv_1407.jpg'])]
# cv2 = cv2[~cv2['_file'].isin(['gsv_52.jpg', 'gsv_1075.jpg', 'gsv_1343.jpg', 'gsv_1407.jpg'])]
# cv3 = cv3[~cv3['_file'].isin(['gsv_52.jpg', 'gsv_1075.jpg', 'gsv_1343.jpg', 'gsv_1407.jpg'])]
# cv4 = cv4[~cv4['_file'].isin(['gsv_52.jpg', 'gsv_1075.jpg', 'gsv_1343.jpg', 'gsv_1407.jpg'])]
# full = full[~full['_file'].isin(['gsv_52.jpg', 'gsv_1075.jpg', 'gsv_1343.jpg', 'gsv_1407.jpg'])]

In [31]:
# cv0.to_csv('/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.hdf5.cv0.prediction.csv', index = False)
# cv1.to_csv('/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.hdf5.cv1.prediction.csv', index = False)
# cv2.to_csv('/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.hdf5.cv2.prediction.csv', index = False)
# cv3.to_csv('/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.hdf5.cv3.prediction.csv', index = False)
# cv4.to_csv('/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.hdf5.cv4.prediction.csv', index = False)
# full.to_csv('/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.wholedata.hdf5.prediction.csv', index = False)

In [43]:
file_list0 = ['/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.hdf5.cv3.prediction.csv',
             '/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.wholedata.hdf5.prediction.csv']


file_list1 = ['/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.hdf5.cv0.prediction.csv',
             '/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.hdf5.cv1.prediction.csv',
             '/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.hdf5.cv2.prediction.csv',
             '/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_bestmodel.hdf5.cv4.prediction.csv',
             '/Users/zhanglingling/Desktop/ML1030/toronto_prediction/toronto_prediction_own_cnn.csv']

In [46]:
# function that calculates conditional ensembling 
def cond_ensemble(file_list0, file_list1):
    
    #produce averaging ensembled model's prediction - 'averaged_prediction'
    df_list0 = []
    for f in file_list0:
        df = pd.read_csv(f)
        df = df.sort_values("_file")
        df_list0.append(df)
        
    prediction0 = pd.concat(df_list0).groupby('_file').mean()
    prediction0.reset_index(level=0, inplace=True)
    prediction0['pred_safety'] =  np.where(prediction0['0'] > 0.5, 0, 1)
    
    
    df_list1 = []
    for f in file_list1:
        df = pd.read_csv(f)
        df = df.sort_values("_file")
        df_list1.append(df)
        
    prediction1 = pd.concat(df_list1).groupby('_file').mean()
    prediction1.reset_index(level=0, inplace=True)
    prediction1['pred_safety'] =  np.where(prediction1['0'] > 0.5, 0, 1)
    
    
    pred = pd.merge(prediction0, prediction1, how='outer', on='_file')
    pred.columns = ['_file', 'pred0_0', 'pred0_1', 'pred0_pred_safety',  'pred1_0', 'pred1_1', 'pred1_pred_safety']

    return pred

In [47]:
pred = cond_ensemble(file_list0, file_list1)
pred['weight_pred_0'] = pred['pred0_0'] * 0.45 + pred['pred1_0'] * 0.55
pred['weight_pred_1'] = pred['pred0_1'] * 0.45 + pred['pred1_1'] * 0.55
pred['pred_safety'] =  np.where(pred['weight_pred_0'] > 0.5, 0, 1)

In [52]:
pred.head()

,_file,pred0_0,pred0_1,pred0_pred_safety,pred1_0,pred1_1,pred1_pred_safety,weight_pred_0,weight_pred_1,pred_safety
0,gsv_0.jpg,0.705739,0.294261,0,0.723045,0.276955,0,0.715257,0.284743,0
1,gsv_1.jpg,0.765972,0.234028,0,0.917020,0.082979,0,0.849049,0.150951,0
2,gsv_10.jpg,0.885248,0.114752,0,0.776663,0.223337,0,0.825527,0.174474,0
3,gsv_100.jpg,0.622889,0.377111,0,0.556393,0.443607,0,0.586316,0.413684,0
4,gsv_1000.jpg,0.210264,0.789736,1,0.391650,0.608350,1,0.310026,0.689974,1


In [53]:
final_pred_toronto = pred[['_file', 'weight_pred_0', 'weight_pred_1', 'pred_safety']]
final_pred_toronto.columns = ['_file', '0', '1', 'pred_safety']
final_pred_toronto.head()

,_file,0,1,pred_safety
0,gsv_0.jpg,0.715257,0.284743,0
1,gsv_1.jpg,0.849049,0.150951,0
2,gsv_10.jpg,0.825527,0.174474,0
3,gsv_100.jpg,0.586316,0.413684,0
4,gsv_1000.jpg,0.310026,0.689974,1


#### Among the 2034 randomly selected Toronto images, 1031 were predicted as 0 (less safe), and 1003 were predicted as 1 (more safe)

In [59]:
final_pred_toronto.groupby('pred_safety').count()['_file']

pred_safety
0    1031
1    1003
Name: _file, dtype: int64

In [54]:
final_pred_toronto.to_csv("/Users/zhanglingling/Desktop/ML1030/final_pred_toronto.csv", index=False)